In [13]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wheat-plant-diseases


In [15]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import classification_report

In [16]:
BASE_PATH = '/kaggle/input/wheat-plant-diseases/wheat-plant-diseases/'

DATA_DIR_TRAIN = '/kaggle/input/wheat-plant-diseases/data/train'
DATA_DIR_TEST = '/kaggle/input/wheat-plant-diseases/data/test'
IMAGE_SIZE = (299, 299) 
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5

In [17]:
TRAIN_CLASS_FOLDERS = ['Aphid', 'Blast', 'Mildew', 'Smut', 'Tan spot']
TEST_CLASS_FOLDERS = ['aphid_test', 'blast_test', 'mildew_test', 'smut_test', 'tan_spot_test']
REPORT_TARGET_CLASSES = ['Aphid', 'Wheat Blast', 'Powdery Mildew', 'Smut', 'Spot Blotch']

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)



In [19]:
# TRAINING GENERATOR
print("--- Initializing Data Generators ---")
try:
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='training', shuffle=True
    )
    # VALIDATION GENERATOR
    validation_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='validation', shuffle=False
    )
    # TEST GENERATOR
    test_generator = test_datagen.flow_from_directory(
        DATA_DIR_TEST, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TEST_CLASS_FOLDERS, class_mode='categorical', shuffle=False
    )
    
    print(f"Training Samples: {train_generator.n}")
    print(f"Validation Samples: {validation_generator.n}")
    print(f"Test Samples: {test_generator.n}")

except Exception as e:
    print(f"\nERROR: Could not load data generators. Check the paths and folder names.")
    print(f"Details: {e}")
    exit()

--- Initializing Data Generators ---
Found 3770 images belonging to 5 classes.
Found 941 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Training Samples: 3770
Validation Samples: 941
Test Samples: 250


In [20]:
print("\n--- Building and Training InceptionV3 Model ---")

# 1. Load Base Model (InceptionV3)
base_model_inception = InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)


total_layers = len(base_model_inception.layers)
UNFREEZE_LAYERS_FROM = total_layers - 40 

for layer in base_model_inception.layers:
    layer.trainable = True 

for layer in base_model_inception.layers[:UNFREEZE_LAYERS_FROM]:
    layer.trainable = False 


x = base_model_inception.output
x = GlobalAveragePooling2D()(x) 
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model_inception.input, outputs=predictions)


--- Building and Training InceptionV3 Model ---
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [21]:
# 3. Compile Model
model.compile(
    
    optimizer=Adam(learning_rate=0.00001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,854,437 (87.18 MB)

 Trainable params: 7,125,381 (27.18 MB)

 Non-trainable params: 15,729,056 (60.00 MB)

In [22]:
# 4. Train Model
start_time = time.time()
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1
)
training_time = time.time() - start_time



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1765008657.787911     181 service.cc:148] XLA service 0x7dbda4016ce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765008657.788840     181 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765008657.788861     181 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765008660.123361     181 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/117 ━━━━━━━━━━━━━━━━━━━━ 52:43 27s/step - accuracy: 0.2188 - loss: 1.8527

I0000 00:00:1765008673.563383     181 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.3312 - loss: 1.5354 - val_accuracy: 0.6121 - val_loss: 1.0599
Epoch 2/15
  1/117 ━━━━━━━━━━━━━━━━━━━━ 14s 125ms/step - accuracy: 0.6250 - loss: 1.0859

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


117/117 ━━━━━━━━━━━━━━━━━━━━ 22s 190ms/step - accuracy: 0.6250 - loss: 1.0859 - val_accuracy: 0.6218 - val_loss: 1.0527
Epoch 3/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 108s 927ms/step - accuracy: 0.6411 - loss: 0.9642 - val_accuracy: 0.7069 - val_loss: 0.8322
Epoch 4/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 22s 190ms/step - accuracy: 0.7812 - loss: 0.8612 - val_accuracy: 0.7004 - val_loss: 0.8465
Epoch 5/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 110s 939ms/step - accuracy: 0.7167 - loss: 0.7832 - val_accuracy: 0.7446 - val_loss: 0.7397
Epoch 6/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 22s 191ms/step - accuracy: 0.7188 - loss: 0.6659 - val_accuracy: 0.7338 - val_loss: 0.7324
Epoch 7/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 110s 937ms/step - accuracy: 0.7643 - loss: 0.6714 - val_accuracy: 0.7371 - val_loss: 0.6937
Epoch 8/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 22s 191ms/step - accuracy: 0.8750 - loss: 0.5553 - val_accuracy: 0.7511 - val_loss: 0.6935
Epoch 9/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 108s 926ms/step - accuracy: 0.7779 - loss: 0.6153 -

In [23]:
# 5. Evaluate and Report
print(f"\n--- VGG19 FINAL RESULTS ---")

# Evaluate on Test Generator
loss, accuracy = model.evaluate(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


--- VGG19 FINAL RESULTS ---
Test Accuracy: 0.8560
Training Time: 1125.80 seconds


In [24]:
# Generate Classification Report
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

report = classification_report(
    y_true, 
    y_pred, 
    target_names=REPORT_TARGET_CLASSES, 
    zero_division=0,
    output_dict=True 
)

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=REPORT_TARGET_CLASSES, zero_division=0))


--- CLASSIFICATION REPORT ---
                precision    recall  f1-score   support

         Aphid       0.82      0.84      0.83        50
   Wheat Blast       0.96      0.98      0.97        50
Powdery Mildew       0.90      0.76      0.83        50
          Smut       0.84      0.96      0.90        50
   Spot Blotch       0.76      0.74      0.75        50

      accuracy                           0.86       250
     macro avg       0.86      0.86      0.85       250
  weighted avg       0.86      0.86      0.85       250



In [25]:
model_name = "InceptionV3"

metrics_summary = {
    'Model': model_name,
    'Accuracy': report['accuracy'],
    'Precision': report['weighted avg']['precision'], 
    'Recall': report['weighted avg']['recall'],
    'F1-Score': report['weighted avg']['f1-score'],
    'Training Time (s)': training_time,
}

df_metrics = pd.DataFrame([metrics_summary])


In [26]:
results_file = 'model_performance_summary.csv'
if not os.path.exists(results_file):
    df_metrics.to_csv(results_file, index=False)
else:
    df_metrics.to_csv(results_file, mode='a', header=False, index=False)
    
print(f"\nMetrics saved to {results_file}")


Metrics saved to model_performance_summary.csv
